In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder\
            .master("local[4]")\
            .appName("hive-support")\
            .config("spark.sql.warehouse.dir","C:/Users/Avinash Godbole/Desktop/Programm File/pyspark_code/youtube_manish_kumar/spark_warehouse")\
            .enableHiveSupport()\
            .getOrCreate()
spark

In [29]:
from pyspark.sql.functions import count,min, max , avg , sum

In [4]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [20]:
schema = "id int, name string, age int, salary int, location string, department string"

df = spark.read.format("csv")\
        .option("header","false")\
        .schema(schema)\
        .load("./emp.csv")

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- department: string (nullable = true)



In [21]:
df.show()

+----+-------+----+------+--------+-----------+
|  id|   name| age|salary|location| department|
+----+-------+----+------+--------+-----------+
|   1| manish|  26| 20000|   india|         IT|
|   2|  rahul|null| 40000| germany|engineering|
|   3|  pawan|  12| 60000|   india|      sales|
|   4|roshini|  44|  null|      uk|engineering|
|   5|raushan|  35| 70000|   india|      sales|
|   6|   None|  29|200000|      uk|         IT|
|   7|   adam|  37| 65000|      us|         IT|
|   8|  chris|  16| 40000|      us|      sales|
|null|   None|null|  null|    None|       None|
|   7|   adam|  37| 65000|      us|         IT|
+----+-------+----+------+--------+-----------+



In [25]:
# count, min ,max, avg

df.select(count("id")).show()


+---------+
|count(id)|
+---------+
|        9|
+---------+



In [33]:
df.select(sum("salary").alias("total"),max("salary"),min("salary"),avg("salary").cast("Int").alias("average")).show()

+------+-----------+-----------+-------+
| total|max(salary)|min(salary)|average|
+------+-----------+-----------+-------+
|560000|     200000|      20000|  70000|
+------+-----------+-----------+-------+



In [35]:
df.groupBy("department").count().alias("total").show()

+-----------+-----+
| department|count|
+-----------+-----+
|       None|    1|
|      sales|    3|
|         IT|    4|
|engineering|    2|
+-----------+-----+



In [37]:
df.groupBy("department").agg(sum("salary"),min("salary"),max("salary"),avg("salary")).show()

+-----------+-----------+-----------+-----------+------------------+
| department|sum(salary)|min(salary)|max(salary)|       avg(salary)|
+-----------+-----------+-----------+-----------+------------------+
|       None|       null|       null|       null|              null|
|      sales|     170000|      40000|      70000|56666.666666666664|
|         IT|     350000|      20000|     200000|           87500.0|
|engineering|      40000|      40000|      40000|           40000.0|
+-----------+-----------+-----------+-----------+------------------+



In [38]:
from pyspark.sql.window import Window

In [39]:
df.groupBy("department").agg(sum("salary")).show()
df.groupBy

+-----------+-----------+
| department|sum(salary)|
+-----------+-----------+
|       None|       null|
|      sales|     170000|
|         IT|     350000|
|engineering|      40000|
+-----------+-----------+



In [40]:
emp_df = spark.read.format("csv")\
            .option("header",True)\
            .option("inferSchema",True)\
            .load("./emp2.csv")

emp_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- dept: string (nullable = true)
 |-- country: string (nullable = true)



In [41]:
emp_df.groupBy("dept").agg(sum("salary")).show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|marketing|     220000|
|    sales|     150000|
|       IT|     295000|
+---------+-----------+



In [42]:
emp_df.groupBy("dept","country").agg(sum("salary")).show()

+---------+-------+-----------+
|     dept|country|sum(salary)|
+---------+-------+-----------+
|       IT|  india|     115000|
|marketing|  india|     125000|
|    sales|     us|      60000|
|marketing|     us|      95000|
|    sales|  india|      90000|
|       IT|     us|     180000|
+---------+-------+-----------+



In [45]:
emp_df.createTempView("emp")

spark.sql("""
            select dept, sum(salary) from emp group by dept
""").show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|marketing|     220000|
|    sales|     150000|
|       IT|     295000|
+---------+-----------+



In [52]:
# join 

customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]

customer_schema=['customer_id','customer_name','address','date_of_joining']


sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']


product_data = [(1, 'fanta',20),
(2, 'dew',22),
(5, 'sprite',40),
(7, 'redbull',100),
(12,'mazza',45),
(22,'coke',27),
(25,'limca',21),
(27,'pepsi',14),
(56,'sting',10)]

product_schema=['id','name','price']

customer_df = spark.createDataFrame(customer_data,customer_schema)

sales_df = spark.createDataFrame(sales_data,sales_schema)

product_df = spark.createDataFrame(product_data,product_schema)

customer_df.printSchema()
sales_df.printSchema()
product_df.printSchema()



root
 |-- customer_id: long (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- date_of_joining: string (nullable = true)

root
 |-- customer_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- date_of_purchase: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- price: long (nullable = true)



In [55]:
customer_df.show()
sales_df.show()
product_df.show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          1|       manish|  patna|     30-05-2022|
|          2|       vikash|kolkata|     12-03-2023|
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|
|          6|     prantosh|kolkata|     18-10-2022|
|          7|        raman|  patna|     30-12-2022|
|          8|      prakash| ranchi|     24-02-2023|
|          9|       ragini|kolkata|     03-03-2023|
|         10|      raushan| jaipur|     05-02-2023|
+-----------+-------------+-------+---------------+

+-----------+----------+--------+----------------+
|customer_id|product_id|quantity|date_of_purchase|
+-----------+----------+--------+----------------+
|          1|        22|      10|      01-06-2022|
|          1|        27|       5|      03-02-2023|
|          2|   

In [57]:
# inner
customer_df.join(sales_df,customer_df["customer_id"]==sales_df["customer_id"],"inner").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15-04-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25

In [60]:
# type of join in spark 
# inner, outer, left join, right join, left semi join, left anti join, cross join

# outer 
customer_df.join(sales_df, customer_df.customer_id==sales_df.customer_id,"left").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          6|     prantosh|kolkata|     18-10-2022|       null|      null|    null|            null|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15-04-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03

In [61]:
customer_df.join(sales_df, customer_df.customer_id==sales_df.customer_id,"right").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15-04-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25

In [62]:
# full outer dimentsion (scd 1 type)
customer_df.join(sales_df, customer_df.customer_id==sales_df.customer_id,"outer").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          6|     prantosh|kolkata|     18-10-2022|       null|      null|    null|            null|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15-04-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03

In [63]:
# left semi join 
customer_df.join(sales_df, customer_df.customer_id==sales_df.customer_id,"left_semi").show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          7|        raman|  patna|     30-12-2022|
|          9|       ragini|kolkata|     03-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|
|          1|       manish|  patna|     30-05-2022|
|          2|       vikash|kolkata|     12-03-2023|
+-----------+-------------+-------+---------------+



In [64]:
# left anti join
customer_df.join(sales_df, customer_df.customer_id==sales_df.customer_id,"left_anti").show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          6|     prantosh|kolkata|     18-10-2022|
|         10|      raushan| jaipur|     05-02-2023|
|          3|       nikita|  delhi|     25-06-2023|
|          8|      prakash| ranchi|     24-02-2023|
|          4|        rahul| ranchi|     24-03-2023|
+-----------+-------------+-------+---------------+



In [66]:
# cross join
customer_df.crossJoin(sales_df).show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          2|       vikash|kolkata|     12-03-2023|          1|        22|      10|      01-06-2022|
|          2|       vikash|kolkata|     12-03-2023|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          2|         5|       3|      01-06-2023|
|          1|       manish|  patna|     30-05-2022|          5|        22|       1|      22-03-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01

In [68]:
customer_df.crossJoin(sales_df).count()

100

In [71]:
# optimization of join
# window
widnow_spec = Window.partitionBy("dept")

emp_df.withColumn("sum_sal",avg("salary").over(widnow_spec)).show()


+---+-------+------+---------+-------+-------+
| id|   name|salary|     dept|country|sum_sal|
+---+-------+------+---------+-------+-------+
|  3|raushan| 70000|marketing|  india|55000.0|
|  6| nikita| 45000|marketing|     us|55000.0|
|  7| ragini| 55000|marketing|  india|55000.0|
| 10|  rahul| 50000|marketing|     us|55000.0|
|  2| vikash| 60000|    sales|     us|75000.0|
|  5| pritam| 90000|    sales|  india|75000.0|
|  1| manish| 50000|       IT|  india|73750.0|
|  4| mukesh| 80000|       IT|     us|73750.0|
|  8| rakesh|100000|       IT|     us|73750.0|
|  9| aditya| 65000|       IT|  india|73750.0|
+---+-------+------+---------+-------+-------+



In [72]:
from pyspark.sql.functions import row_number, rank, dense_rank

In [74]:
# optimization of join
# window
window_spec = Window.partitionBy("dept").orderBy("salary")

emp_df.withColumn("row_number",row_number().over(window_spec))\
    .withColumn("dense_rnk",dense_rank().over(window_spec))\
        .withColumn("rnk",rank().over(window_spec)).show()

+---+-------+------+---------+-------+----------+---------+---+
| id|   name|salary|     dept|country|row_number|dense_rnk|rnk|
+---+-------+------+---------+-------+----------+---------+---+
|  6| nikita| 45000|marketing|     us|         1|        1|  1|
| 10|  rahul| 50000|marketing|     us|         2|        2|  2|
|  7| ragini| 55000|marketing|  india|         3|        3|  3|
|  3|raushan| 70000|marketing|  india|         4|        4|  4|
|  2| vikash| 60000|    sales|     us|         1|        1|  1|
|  5| pritam| 90000|    sales|  india|         2|        2|  2|
|  1| manish| 50000|       IT|  india|         1|        1|  1|
|  9| aditya| 65000|       IT|  india|         2|        2|  2|
|  4| mukesh| 80000|       IT|     us|         3|        3|  3|
|  8| rakesh|100000|       IT|     us|         4|        4|  4|
+---+-------+------+---------+-------+----------+---------+---+



In [75]:
spark.stop()